Create S3 bucket

In [1]:
import boto3
s3 = boto3.resource('s3')

**Boto3 is a AWS SDK **

In [4]:
region = 'eu-north-1'

# Create an S3 client with the specified region
s3 = boto3.client('s3', region_name=region)

In [7]:
bucket_name = 'yahoofinancestockprice-1'
try:
    # Create the S3 bucket with the specified region
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
    print('S3 bucket has been created')
except Exception as e:
    print('S3 error', e)
    

S3 bucket has been created


Create train and validate CSV

In [8]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 80.8 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 93.0 MB/s eta 0:00:00:00:0100:01
  Created wheel for peewee: filename=peewee-3.17.0-cp310-cp310-linux_x86_64.whl size=275946 sha256=cce7b0ea0b468fa697a3c976af561b4e6df77079d8094065d5bceaa4b5267fc9
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c7/70/ad/212867e96e7004265a69c4aa5dcff00a95f547a67ba26e7e76
Successfully built peewee


In [10]:
import pandas as pd
from datetime import datetime
import yfinance as yf

#Initialize the start and end date
start_date = datetime(2019,1,1)
end_date = datetime(2024,1,1)

#get the data
df_data = yf.download('IIF', start = start_date, end= end_date)

df_data.reset_index(inplace=True)

df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,19.910000,19.969999,19.760000,19.910000,14.000065,59800
1,2019-01-03,19.780001,19.780001,19.520000,19.600000,13.782083,40800
2,2019-01-04,19.830000,20.580000,19.809999,20.500000,14.414933,19700
3,2019-01-07,20.190001,20.190001,19.959999,20.150000,14.168824,97500
4,2019-01-08,19.990000,20.320000,19.990000,20.309999,14.281331,25800
...,...,...,...,...,...,...,...
1253,2023-12-22,21.389999,21.520000,21.230000,21.360001,21.360001,61200
1254,2023-12-26,21.570000,21.570000,21.350000,21.379999,21.379999,37500
1255,2023-12-27,21.420000,21.520000,21.420000,21.469999,21.469999,32500
1256,2023-12-28,21.629999,21.780001,21.500000,21.549999,21.549999,47500


Etract , Load and Transform the data

In [11]:
#Dropping the columns
df_data.drop(axis=1, columns=['Adj Close'], inplace=True)
df_data.drop(axis=1, columns=['Date'], inplace = True)

In [13]:
#Extract the features
df_data_features = df_data.iloc[: -1 , :]
df_data_features

,Open,High,Low,Close,Volume
0,19.910000,19.969999,19.760000,19.910000,59800
1,19.780001,19.780001,19.520000,19.600000,40800
2,19.830000,20.580000,19.809999,20.500000,19700
3,20.190001,20.190001,19.959999,20.150000,97500
4,19.990000,20.320000,19.990000,20.309999,25800
...,...,...,...,...,...
1252,21.080000,21.290001,21.080000,21.230000,47900
1253,21.389999,21.520000,21.230000,21.360001,61200
1254,21.570000,21.570000,21.350000,21.379999,37500
1255,21.420000,21.520000,21.420000,21.469999,32500


In [14]:
#Extract only the open price from the 2nd day(i.e the next day) for the prediction
df_data_target = df_data.iloc[1:,0].rename('Target')
df_data_target

1       19.780001
2       19.830000
3       20.190001
4       19.990000
5       20.240000
          ...    
1253    21.389999
1254    21.570000
1255    21.420000
1256    21.629999
1257    21.559999
Name: Target, Length: 1257, dtype: float64

In [15]:
#Final data
df_data_features['Target'] = list(df_data_target)

first_col = df_data_features.pop('Target')
df_data_features.insert(0,'Target', first_col)

df_data_final = df_data_features

df_data_final


/tmp/ipykernel_8574/1034691346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_features['Target'] = list(df_data_target)


,Target,Open,High,Low,Close,Volume
0,19.780001,19.910000,19.969999,19.760000,19.910000,59800
1,19.830000,19.780001,19.780001,19.520000,19.600000,40800
2,20.190001,19.830000,20.580000,19.809999,20.500000,19700
3,19.990000,20.190001,20.190001,19.959999,20.150000,97500
4,20.240000,19.990000,20.320000,19.990000,20.309999,25800
...,...,...,...,...,...,...
1252,21.389999,21.080000,21.290001,21.080000,21.230000,47900
1253,21.570000,21.389999,21.520000,21.230000,21.360001,61200
1254,21.420000,21.570000,21.570000,21.350000,21.379999,37500
1255,21.629999,21.420000,21.520000,21.420000,21.469999,32500


**Train Test SPlit **

In [16]:
import numpy as np

df_randomized = df_data_final.sample(frac=1, random_state=123)

df_randomized

,Target,Open,High,Low,Close,Volume
870,21.920000,22.320000,22.510000,22.200001,22.379999,18400
367,15.480000,15.120000,15.450000,15.110000,15.410000,90500
875,22.010000,21.770000,21.990000,21.670000,21.900000,13200
525,22.850000,22.570000,22.760000,22.510000,22.750000,14900
1256,21.559999,21.629999,21.780001,21.500000,21.549999,47500
...,...,...,...,...,...,...
1238,23.120001,22.660000,22.969999,22.660000,22.950001,43100
1147,22.450001,22.360001,22.450001,22.350000,22.400000,39500
106,21.230000,21.570000,21.570000,20.980000,21.290001,48900
1041,20.040001,20.190001,20.190001,20.090000,20.090000,18800


In [17]:
train_data, test_data = np.split(df_randomized, [int(0.8*len(df_randomized))])
print(train_data.shape, test_data.shape)

(1005, 6) (252, 6)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Set a path upload data to S3 bucket

In [18]:
import os

prefix = 'xgboost-as-a-built-in-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix,'train','train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix,'test','test.csv')

print(train_csv_path)
print(test_csv_path)

s3://yahoofinancestockprice-1/xgboost-as-a-built-in-algo/train/train.csv
s3://yahoofinancestockprice-1/xgboost-as-a-built-in-algo/test/test.csv


In [19]:
train_data.to_csv(train_csv_path, index= False, header= False)
test_data.to_csv(test_csv_path, index= False, header= False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


**BUILD XGBOOST MODEL**

In [20]:
#XGBOOST as a built in algo
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Find Xgboost image URI adn build an Xgboost cotainer

In [25]:
xgboost_container = image_uris.retrieve("xgboost", region,"1.5-1")

display(xgboost_container)

'662702820516.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-xgboost:1.5-1'

**INitialize Hyperparameters**

In [28]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "verbosity":"1",
        "objective":"reg:squarederror",
        "early_stopping_rounds" : 10,
        "num_round":50}

Output path to save the trained model

In [30]:
output_path = 's3://{}/{}/{}/'.format(bucket_name, prefix,'output')

print(output_path)

s3://yahoofinancestockprice-1/xgboost-as-a-built-in-algo/output/


Build a sagemaker estimator that calls Xgboost container

In [31]:
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances = True, #wait until the next instance to some
                                          max_run = 300,
                                          max_wait = 600
                                         )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Defining data typea and paths for the training and validation dataset which was saved earlier

In [32]:
content_type ='csv'

train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'train'), content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'test'), content_type=content_type)

Execution of Xgboost algo

In [34]:
# execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-01-04-19-24-47-568


2024-01-04 19:24:47 Starting - Starting the training job...
2024-01-04 19:25:02 Starting - Preparing the instances for training.........
2024-01-04 19:26:21 Downloading - Downloading input data...
2024-01-04 19:27:06 Training - Training image download completed. Training in progress.../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2024-01-04 19:27:28.159 ip-10-0-245-0.eu-north-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-01-04 19:27:28.180 ip-10-0-245-0.eu-north-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-01-04:19:27:28:INFO] Imported framework sagemaker_xgboost_container.training
[2024-01-04:19:27:28:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the va

**Deploy the trained xgb model as Endpoint

In [35]:
from sagemaker.serializers import CSVSerializer

xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type = 'ml.m5.2xlarge', serializer = CSVSerializer()) #iic=1, the number of instance deployed ,,, serializer - serialize input data of various formats to a cSV format coz xgboots accepts only tat

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-01-05-06-40-57-245
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-01-05-06-40-57-245
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-01-05-06-40-57-245


----!

In [36]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2024-01-05-06-40-57-245'

**Make predictions with the use of Endpoints

In [37]:
start_date = datetime(2024,1,2)
end_date = datetime(2024,1,3)

#get the data
df_data = yf.download('IIF', start = start_date, end= end_date)

df_data.reset_index(inplace=True)

df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-01-02,21.459999,21.59,21.379999,21.4,21.4,68200


In [38]:
df_data.drop(axis=1, columns=['Adj Close'], inplace=True)
df_data.drop(axis=1, columns=['Date'], inplace = True)

data_features_array = df_data.values
data_features_array 

array([[2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04]])

**Serialize Data**

In [39]:
#serialize input by sagemaker function
y_pred_fn = xgb_predictor.predict(data_features_array).decode('utf-8') #utf-8 to convert bytes to string
y_pred_fn

'21.51511573791504\n'

In [ ]:
#OR the same above code can also be written as
# from sagemaker.serialzer import CSVSErializer

###Serialized_ip_fn = CSVSerializer().serailize([[2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
#        6.82000000e+04]])
# print(Serialized_ip_fn)

#y_pred_fn = xgb_predictor.predict(Serialized_ip_fn).decode('utf-8') #utf-8 to convert bytes to string
#y_pred_fn
#it would give the same output of 21.5...

In [41]:
#serialize input by built in fn(Lambda fn)

Input = [[2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04],
        [2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04],
        [2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04]]

serialized_input = ','.join(map(str, Input[0])) #take the first element from the list as str and join them with the ,

print(serialized_input, type(serialized_input))

y_pred_fn = xgb_predictor.predict(serialized_input).decode('utf-8') #utf-8 to convert bytes to string
y_pred_fn

21.4599991,21.5900002,21.3799992,21.3999996,68200.0 <class 'str'>


'21.51511573791504\n'

**Lambda function to invoke the endpoint
1. trigger Endpoint
2. Trigger SNS
With lambda you can run code virtually any type of application or backend service

**Lambda function handler

In [46]:
import boto3

ENDPOINT_NAME = 'sagemaker-xgboost-2024-01-05-06-40-57-245' #We got this in the previous cells
runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    
    inputs = event['data']
    result = []
    
    for input in inputs:
        serialized_input = ','.join(map(str, input)) #take the first element from the list as str and join them with the ,
    
        response = runtime.invoke_endpoint(EndpointName= ENDPOINT_NAME,
                                      ContentType = 'text/csv',
                                      Body= serialized_input)
    
        result.append(response['Body'].read().decode())
    
    return result

Input_json = {'data':
             [[2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04],
        [2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04],
        [2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04]]
             }

result = lambda_handler(Input_json, _)
result

['21.51511573791504\n', '21.51511573791504\n', '21.51511573791504\n']

**Post Request

In [47]:
import requests

API_ENDPOINT = "https://qr94phmnc3.execute-api.us-west-2.amazonaws.com/Xgboostmodel"

json = {"data":[[2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04],
        [2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04],
        [2.14599991e+01, 2.15900002e+01, 2.13799992e+01, 2.13999996e+01,
        6.82000000e+04]]
       }

r = requests.post(url = API_ENDPOINT,  json =json)

In [48]:
print(f"Status Code :{r.status_code}, Response : {r.json()} " )

Status Code :200, Response : ['21.51511573791504\n', '21.51511573791504\n', '21.51511573791504\n'] 


**Close and Terminate

In [ ]:
#sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

In [ ]:
#bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
#bucket_to_delete.objects.all().delete()